In [1]:
!pip install numpy

     |████████████████████████████████| 17.1 MB 7.3 MB/s eta 0:00:01█████▍                        | 4.0 MB 7.3 MB/s eta 0:00:02


In [9]:
from pathlib import Path
import numpy as np

In [2]:
def compute_PRAF(ytrue,ypred,threshold=0.5):
	# Fake positives
	FP = 0
	# Fake negatives
	FN = 0
	# True positives
	TP = 0
	# True negatives
	TN = 0

	for idx in range(len(ytrue)):
		if (ypred[idx]>=threshold) and (ytrue[idx]>=threshold):
			TP = TP + 1
		elif (ypred[idx]>=threshold) and (ytrue[idx]<threshold):
			FP = FP + 1
		elif (ypred[idx]<threshold) and (ytrue[idx]>=threshold):
			FN = FN + 1
		elif (ypred[idx]<threshold) and (ytrue[idx]<threshold):
			TN = TN + 1

	precision = 1.0
	recall = 1.0

	if (TP+FP) > 0:
		precision = float(TP)/float(TP+FP)

	if (TP+FN) > 0:
		recall = float(TP)/float(TP+FN)

	accuracy = float(TP+TN)/float(TP+TN+FP+FN)
	
	if precision+recall > 0:
		F = 2*((precision*recall)/(precision+recall))
	else:
		F = 0.0

	return accuracy,precision,recall,F

In [8]:
DATA_PATH = Path.cwd() / 'work' / 'data' / 'CNN5a'

[f for f in DATA_PATH.iterdir()]

[PosixPath('/home/jovyan/work/data/CNN5a/1'),
 PosixPath('/home/jovyan/work/data/CNN5a/10'),
 PosixPath('/home/jovyan/work/data/CNN5a/2'),
 PosixPath('/home/jovyan/work/data/CNN5a/3'),
 PosixPath('/home/jovyan/work/data/CNN5a/4'),
 PosixPath('/home/jovyan/work/data/CNN5a/5'),
 PosixPath('/home/jovyan/work/data/CNN5a/6'),
 PosixPath('/home/jovyan/work/data/CNN5a/7'),
 PosixPath('/home/jovyan/work/data/CNN5a/8'),
 PosixPath('/home/jovyan/work/data/CNN5a/9')]

In [50]:
def get_fold_praf(fold_number, set_name):
    n_pullbacks = 8
    fold_folder_path = DATA_PATH / str(fold_number) / 'CSV' / set_name
    fold_metrics = np.zeros((8, 3))

    for index, pullback_path in enumerate(fold_folder_path.iterdir()):
        data = np.loadtxt(pullback_path, delimiter=',')
        y_pred = data[:, 0]
        y_real = data[:, 1]
        _, pre, rec, f_1 = compute_PRAF(y_real, y_pred)
        fold_metrics[index, 0] = f_1
        fold_metrics[index, 1] = pre
        fold_metrics[index, 2] = rec

    return np.median(fold_metrics, axis=0)

In [51]:
val_1 = get_fold_praf(1, 'VAL')

In [52]:
def display_values(values):
    return [
        '%.2f'%value
        for value in values
    ]

In [53]:
def calculate_metrics(set_name):
    all_praf = [
        get_fold_praf(fold_number, set_name)
        for fold_number in range(1, 11)
    ]

    means = np.mean(all_praf, axis=0)
    stds = np.std(all_praf, axis=0)

    return (display_values(means), display_values(stds))
        

In [54]:
calculate_metrics('VAL')

(['0.58', '0.88', '0.53'], ['0.11', '0.14', '0.19'])

In [55]:
calculate_metrics('TEST')

(['0.58', '0.89', '0.56'], ['0.17', '0.11', '0.22'])